In [122]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import utils

torch.manual_seed(42069)
# if gpu available else cpu
device = torch.device(0) if torch.cuda.is_available() else torch.device('cpu')
TRAIN_MEAN = [0.5036, 0.4719, 0.3897]
TRAIN_STD = [0.2623, 0.2577, 0.2671]
classes = ['butterfly','cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']

## Przygotowanie danych

In [123]:
# zdefiniowanie operacji na kazdym obrazie w zbiorze
transform = transforms.Compose([
    transforms.RandomResizedCrop(64),  #  na 256x256
    transforms.RandomHorizontalFlip(),  # wycinamy losowy fragment 128x128
    transforms.ToTensor(),              # obrazy zamieniamy na tensory,
    # srednie i odchylenia po kanałach całęgo zbioru,
    #  wyliczone wczesniej za pomocą utils.data_normalize_values
    transforms.Normalize(TRAIN_MEAN, TRAIN_STD)
])

# loader danych z batchami
train_data = ImageFolder(root='dataset/train/', transform=transform)
test_data = ImageFolder(root='dataset/test/', transform=transform)
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [124]:
class LeNet(nn.Module):
    def __init__(self, num_classes=len(classes)):
        super(LeNet, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=3),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=128, kernel_size=5, padding=3),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=4),
            
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=256, out_features=32),
            nn.ReLU(inplace=True),
        )
        self.fc = nn.Linear(in_features=32, out_features=num_classes)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return self.fc(x)

## Uczenie klasyfikatora

In [125]:
# RESNET18 z wagami przetrenowanymi na zbiorze IMAGENET
le_net = LeNet()
# inicjalizacja wag w warstwie wyjsciowej
for name, params in le_net.named_parameters():
    if params.ndim>2:
        nn.init.xavier_uniform_(params)
        
le_net = le_net.to(device)
metrics = utils.train_fine_tuning(
    model = le_net, 
    learning_rate= 5e-4, 
    train_loader=train_loader,
    test_loader=test_loader,
    device=device, num_epochs=15, param_group=False
)

Progress: 1/15 epochs
Epoch: 1, Loss 2.211, Train acc: 0.199, Test acc: 0.223
Progress: 2/15 epochs
Epoch: 2, Loss 2.156, Train acc: 0.234, Test acc: 0.232
Progress: 3/15 epochs
Epoch: 3, Loss 2.112, Train acc: 0.255, Test acc: 0.26
Progress: 4/15 epochs
Epoch: 4, Loss 2.066, Train acc: 0.272, Test acc: 0.282
Progress: 5/15 epochs
Epoch: 5, Loss 2.012, Train acc: 0.289, Test acc: 0.295
Progress: 6/15 epochs
Epoch: 6, Loss 1.959, Train acc: 0.31, Test acc: 0.279
Progress: 7/15 epochs
Epoch: 7, Loss 1.905, Train acc: 0.325, Test acc: 0.342
Progress: 8/15 epochs
Epoch: 8, Loss 1.86, Train acc: 0.348, Test acc: 0.352
Progress: 9/15 epochs
Epoch: 9, Loss 1.816, Train acc: 0.364, Test acc: 0.368
Progress: 10/15 epochs
Epoch: 10, Loss 1.784, Train acc: 0.375, Test acc: 0.394
Progress: 11/15 epochs
Epoch: 11, Loss 1.741, Train acc: 0.393, Test acc: 0.392
Progress: 12/15 epochs
Epoch: 12, Loss 1.718, Train acc: 0.404, Test acc: 0.395
Progress: 13/15 epochs
Epoch: 13, Loss 1.678, Train acc: 0.41